## Get predictions using Kipoi embedded model

We need (at least):
- dl_kwargs = gtf_file, fasta_file, vcf_file
- output file = predictions.tsv'

In [14]:
### Standalone script
import kipoi
from kipoi_cadd.writers import LmdbWriter
from kipoi_cadd.utils import load_pickle
from kipoi_cadd.data_utils import calculate_map_size
import kipoi_veff.snv_predict as sp
from kipoi_veff.scores import Logit
from kipoi import get_model
import os

training_dir_hg37 = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37"
gtf_file = "/s/genomes/human/hg38/Homo_sapiens.GRCh38.79.chr-prepend.gtf"
# gtf_file = "/s/genomes/human/hg19/gencode28/gencode.v28.annotation.gff3.gz"
fasta_file = "/s/genomes/human/hg38/hg38.fa"
vcf_file = os.path.join(training_dir_hg37, "example_10.vcf")
lmdb_mmsplice = os.path.join(training_dir_hg37, "lmdb/MMSplice_deltaLogitPSI")
if not os.path.exists(lmdb_mmsplice):
    os.makedirs(lmdb_mmsplice)

model = get_model("MMSplice/deltaLogitPSI")
dl_kwargs = {'gtf_file': gtf_file, 'fasta_file': fasta_file, 'vcf_file': vcf_file}
dataloader = model.default_dataloader
# dl = model.default_dataloader(**dl_kwargs)

In [ ]:
%%time
pred = model.pipeline.predict(dl_kwargs, batch_size=4, num_workers=64)

## Get predictions from their package

In [ ]:
# Import
from mmsplice.vcf_dataloader import SplicingVCFDataloader
from mmsplice import MMSplice, predict_all_table
from mmsplice.utils import max_varEff

# example files
gtf = "/s/genomes/human/hg38/Homo_sapiens.GRCh38.79.chr-prepend.gtf"
vcf = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/example_10.vcf"
fasta = "/s/genomes/human/hg38/hg38.fa"

# dataloader to load variants from vcf
dl = SplicingVCFDataloader(gtf,
                          fasta,
                          vcf,
                          split_seq=False)

# Specify model
model = MMSplice(
    exon_cut_l=0,
    exon_cut_r=0,
    acceptor_intron_cut=6,
    donor_intron_cut=6,
    acceptor_intron_len=50,
    acceptor_exon_len=3,
    donor_exon_len=5,
    donor_intron_len=13)

# Do prediction
predictions = predict_all_table(model, dl, batch_size=1024, split_seq=False, assembly=False)

Using TensorFlow backend.
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator HuberRegressor from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-pa

In [ ]:
# Summerize with maximum effect size
predictionsMax = max_varEff(predictions)